In [1]:
# import sys
# sys.path.extend([
#     '../../openml-python-OpenMLActiveClassificationTask/',
#     '../../openml-scikit-activeml/'
# ])
# %load_ext autoreload
# %autoreload 2

In [2]:
import sys
sys.path.extend([
    'openml-python-OpenMLActiveClassificationTask/',
    'openml-scikit-activeml/'
])
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from sklearn import ensemble
from openml.tasks import OpenMLActiveClassificationTask, TaskType
from openml import tasks, runs
from skactiveml.classifier import ParzenWindowClassifier
from skactiveml.pool import UncertaintySampling, RandomSampling, ProbabilisticAL
import openml
import openml_skactiveml
from openml_skactiveml import PoolSkactivemlModel
openml.config.start_using_configuration_for_example()

c:\Users\tpham\Desktop\Arbeit\openml-skactiveml\openml-python-OpenMLActiveClassificationTask\openml\config.py:184: UserWarning: Switching to the test server https://test.openml.org/api/v1/xml to not upload results to the live server. Using the test server may result in reduced performance of the API!
  warnings.warn(


In [4]:
task_id = 1328
# task_id = 6175
task = tasks.get_task(task_id, download_data=True)

In [5]:
task

OpenML Active Classification Task
Task Type Description: https://test.openml.org/tt/TaskType.ACTIVE_CLASSIFICATION
Task ID..............: 1328
Task URL.............: https://test.openml.org/t/1328
Estimation Procedure.: crossvalidation
Evaluation Measure...: predictive_accuracy
Target Feature.......: class
# of Classes.........: 3
Cost Matrix..........: Available

In [6]:
# # # # Example on how an AL task can be created from an existing classification task

# task_id = 1196
# task = tasks.get_task(task_id)

# my_task = openml.tasks.create_task(
#     task_type=TaskType.SUPERVISED_CLASSIFICATION,
#     dataset_id=128,
#     target_name="class",
#     evaluation_measure="predictive_accuracy",
#     estimation_procedure_id=1,
# )

# task = OpenMLActiveClassificationTask(
#         task_type_id=TaskType.ACTIVE_CLASSIFICATION,
#         task_type="ACTIVE_CLASSIFICATION",
#         data_set_id=task.dataset_id,
#         target_name=task.target_name,
#         class_labels=task.class_labels,
#         evaluation_measure="predictive_accuracy",
#         estimation_procedure_id=30,
#         # annotation_costs=np.arange(150, dtype=float)*0+1, # hard-coded annotation cost for now
#         batch_size=1,
#     )

# task.publish()

In [7]:
from sklearn.ensemble import VotingClassifier
from skactiveml.classifier import SklearnClassifier

model = PoolSkactivemlModel(
    query_strategy=RandomSampling(missing_label=None),
    prediction_model=ParzenWindowClassifier(missing_label=None, classes=task.class_labels),
    # selection_model=ParzenWindowClassifier(missing_label=None, classes=task.class_labels),
    # selection_model_name='clf',
    # extra_query_params={
    # },
    budget=100000,
)

run = runs.run_model_on_task(model, task, upload_flow=False, avoid_duplicate_runs=False, n_jobs=-1, seed=0)

openml==0.13.1.dev,skactiveml==0.4.0,
numpy>=1.17.3
scipy>=1.3.2
joblib>=1.1.1
threadpoolctl>=2.0.0
openml==0.13.1.dev,skactiveml==0.4.0,
numpy>=1.17.3
scipy>=1.3.2
joblib>=1.1.1
threadpoolctl>=2.0.0
,openml==0.13.1.dev,openml_skactiveml==0.0.1,skactiveml==0.4.0,
numpy>=1.17.3
scipy>=1.3.2
joblib>=1.1.1
threadpoolctl>=2.0.0


In [8]:
run

OpenML Run
Uploader Name: None
Metric.......: None
Run ID.......: None
Task ID......: 1328
Task Type....: None
Task URL.....: https://test.openml.org/t/1328
Flow ID......: None
Flow Name....: openml_skactiveml.extension.PoolSkactivemlModel(prediction_model=skactiveml.classifier._parzen_window_classifier.ParzenWindowClassifier,query_strategy=skactiveml.pool._random_sampling.RandomSampling)
Flow URL.....: https://test.openml.org/f/None
Setup ID.....: None
Setup String.: Python_3.9.16. Sklearn_1.2.2. NumPy_1.24.3. SciPy_1.9.3.
Dataset ID...: 128
Dataset URL..: https://test.openml.org/d/128

In [9]:
run.publish()

openml==0.13.1.dev,skactiveml==0.4.0,
numpy>=1.17.3
scipy>=1.3.2
joblib>=1.1.1
threadpoolctl>=2.0.0
openml==0.13.1.dev,skactiveml==0.4.0,
numpy>=1.17.3
scipy>=1.3.2
joblib>=1.1.1
threadpoolctl>=2.0.0


OpenML Run
Uploader Name: None
Metric.......: None
Run ID.......: 507
Run URL......: https://test.openml.org/r/507
Task ID......: 1328
Task Type....: None
Task URL.....: https://test.openml.org/t/1328
Flow ID......: 1399
Flow Name....: openml_skactiveml.extension.PoolSkactivemlModel(prediction_model=skactiveml.classifier._parzen_window_classifier.ParzenWindowClassifier,query_strategy=skactiveml.pool._random_sampling.RandomSampling)
Flow URL.....: https://test.openml.org/f/1399
Setup ID.....: None
Setup String.: Python_3.9.16. Sklearn_1.2.2. NumPy_1.24.3. SciPy_1.9.3.
Dataset ID...: 128
Dataset URL..: https://test.openml.org/d/128

In [10]:
# from sklearn.ensemble import VotingClassifier
# from skactiveml.classifier import SklearnClassifier

# model = {
#     'query_strategy':UncertaintySampling(missing_label=None),
#     'prediction_model':ParzenWindowClassifier(missing_label=None, classes=task.class_labels),
#     'query_params':{
#         "clf": ParzenWindowClassifier(missing_label=None, classes=task.class_labels)
#     },
#     'budget': 100000,
# }

# # TODO: Dictionary with Baseestimators do not work
# model = PoolSkactivemlModel(
#     query_strategy=ProbabilisticAL(missing_label=None),
#     prediction_model=ParzenWindowClassifier(missing_label=None, classes=task.class_labels),
#     query_params={
#         "clf": ParzenWindowClassifier(missing_label=None, classes=task.class_labels)
#     },
#     # query_params = {},
#     budget=100000,
# )

# model = PoolSkactivemlModel(
#     query_strategy=RandomSampling(missing_label=None),
#     prediction_model=SklearnClassifier(VotingClassifier(estimators=[ParzenWindowClassifier(missing_label=None, classes=task.class_labels), ParzenWindowClassifier(missing_label=None, classes=task.class_labels), ParzenWindowClassifier(missing_label=None, classes=task.class_labels)])),
#     query_params={
#     },
#     # query_params = {},
#     budget=100000,
# )

# model = PoolSkactivemlModel(
#     query_strategy=UncertaintySampling(missing_label=None),
#     prediction_model=ParzenWindowClassifier(missing_label=None, classes=task.class_labels),
#     selection_model=ParzenWindowClassifier(missing_label=None, classes=task.class_labels),
#     selection_model_name='clf',
#     extra_query_params={
#     },
#     budget=100000,
# )

# model = PoolSkactivemlModel(
#     query_strategy=RandomSampling(missing_label=None),
#     prediction_model=ParzenWindowClassifier(missing_label=None, classes=task.class_labels),
#     # selection_model=ParzenWindowClassifier(missing_label=None, classes=task.class_labels),
#     # selection_model_name='clf',
#     # extra_query_params={
#     # },
#     budget=100000,
# )

# model = PoolSkactivemlModel(
#     query_strategy=RandomSampling(missing_label=None),
#     prediction_model=ParzenWindowClassifier(missing_label=None, classes=task.class_labels),
#     selection_model=ParzenWindowClassifier(missing_label=None, classes=None),
#     selection_model_name='clf',
#     extra_query_params={
#     },
#     budget=100000,
# )

# TODO: Dictionary with Baseestimators do not work
# model = PoolSkactivemlModel(
#     query_strategy=RandomSampling(missing_label=None),
#     prediction_model=ParzenWindowClassifier(missing_label=None, classes=task.class_labels),
#     query_params={
#         # "clf": None# ParzenWindowClassifier(missing_label=None, classes=task.class_labels)
#     },
#     # query_params = {},
#     budget=100000,
# )
# Automatically evaluate your model on the task
# run = runs.run_model_on_task(model, task, upload_flow=False, avoid_duplicate_runs=False, n_jobs=-1, seed=0)

In [11]:
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff 

raw_data = loadarff('log')
df_data = pd.DataFrame(raw_data[0])
df_data

,repeat,fold,cycle,budget,row_id,confidence.Iris-setosa,confidence.Iris-versicolor,confidence.Iris-virginica,prediction,correct
0,0.0,0.0,0.0,1.0,43.0,1.000000,0.000000,0.000000,b'Iris-setosa',b'Iris-setosa'
1,0.0,0.0,0.0,1.0,14.0,1.000000,0.000000,0.000000,b'Iris-setosa',b'Iris-setosa'
2,0.0,0.0,0.0,1.0,37.0,1.000000,0.000000,0.000000,b'Iris-setosa',b'Iris-setosa'
3,0.0,0.0,0.0,1.0,23.0,1.000000,0.000000,0.000000,b'Iris-setosa',b'Iris-setosa'
4,0.0,0.0,0.0,1.0,10.0,1.000000,0.000000,0.000000,b'Iris-setosa',b'Iris-setosa'
...,...,...,...,...,...,...,...,...,...,...
20095,0.0,9.0,133.0,134.0,142.0,0.009959,0.498706,0.491335,b'Iris-virginica',b'Iris-versicolor'
20096,0.0,9.0,133.0,134.0,123.0,0.011183,0.507453,0.481364,b'Iris-virginica',b'Iris-versicolor'
20097,0.0,9.0,133.0,134.0,131.0,0.000277,0.208998,0.790726,b'Iris-virginica',b'Iris-virginica'
20098,0.0,9.0,133.0,134.0,147.0,0.005609,0.438550,0.555841,b'Iris-virginica',b'Iris-virginica'


In [12]:
raw_data = loadarff('log_clf')
df_data = pd.DataFrame(raw_data[0])
df_data['sample'].unique()

array([0.])

In [13]:
''

''